In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mlaut.data import Data
from mlaut.estimators.estimators import instantiate_default_estimators
from mlaut.experiments import Orchestrator
from mlaut.analyze_results import AnalyseResults
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mlaut.analyze_results.scores import ScoreAccuracy
import pandas as pd

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Step 1: Download the datasets

For the puposes of this demonstration we will use datasets that can be downloaded from:
https://persoal.citius.usc.es/manuel.fernandez.delgado/papers/jmlr/data.tar.gz

The datasets were used in the study <cite data-cite="delgado2014">Do we Need Hundreds of Classifiers to solve Real World Classification Problems? (Delgado, 2014)</cite> and were originally downloaded from the UCI Machine Learning Repository.

The code below uses this data and creates two arrays. The first one contains the actual datasets and the scond one the metadata associated with each dataset. The metadata should be in the form of a dictionary and must contain at a minimum 

```python
{'class_name': ..., #name of the column containing the labels
 'dataset_name': ... #name of the dataset
}
```

This step is not MLAUT specific but the data needs to be stored in a MLAUT compatible format so that the experiments can be run.

In [2]:
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False)

Error: Dataset Delgado_data/molec-biol-protein-second has a different number of arff files


### Step 2: Store the datasets in MLAUT format

The next step is to store the datasets in a format that can be used in MLAUT.

Interaction with MLAUT's data structures is done though the `Data()` class that serves as interface for storing and loading data into HDF5 database which is used by MLAUT for data maniputlation.

In [3]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='a')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')

We are now ready to store the data in the HDF5 database. 

`save_loc_hdf5` indicates the HDF5 group in which the datasets will be saved.<br>
`datasets` are the actual datasets in pandas format.<br>
`dts_metadata` is the metadata array attached to each dataset.<br>
`input_io` is the class object that interfaces the HDF5 file.

In [4]:
data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, input_io=input_io)

### Step 3: Split datasets

The next step is to split the data in test and training. 

Unless otherwise specified we use $\dfrac{2}{3}$ of the data for training and $\dfrac{1}{3}$ for testing. We do not change or move the original data in this process. Instead we store the train/test indices in a separate HDF5 database.

The code below generates warnings as we have already generated the split.

In [5]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path)

### Step 4: Run the experiments

At this step we need to select the estimators that we want to use in the study. In this example we enumerated the estimators by name. However, MLAUT also supports a search by task or estimator familily.

The user also needs to instantiate the test orchestrator object by providing reference to the input and output database files and the location of the datasets inside the HDF5 database.

The final step is to run the experiments by invoking the `run()` method.

The code below generates warnings as we had already trained the estimators. This step could take a substantial amount of time depending on the number and size of datasets and the number of estimators that we wish to train.

In [7]:
est = ['RandomForestClassifier','BaggingClassifier','GradientBoostingClassifier','SVC','GaussianNaiveBayes','BernoulliNaiveBayes','NeuralNetworkDeepClassifier','PassiveAggressiveClassifier','BaselineClassifier']
estimators = instantiate_default_estimators(estimators=est)
orchest = Orchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io, dts_names=dts_names_list,
                 original_datasets_group_h5_path='delgado_datasets/')
orchest.run(modelling_strategies=estimators)

INFO:root:Estimator RandomForestClassifier already trained on abalone. Skipping it.
2018-09-19 21:50:25,703 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on abalone. Skipping it.
INFO:root:Estimator BaggingClassifier already trained on abalone. Skipping it.
2018-09-19 21:50:25,710 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on abalone. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on abalone. Skipping it.
2018-09-19 21:50:25,714 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on abalone. Skipping it.
INFO:root:Estimator SVC already trained on abalone. Skipping it.
2018-09-19 21:50:25,719 [MainThread  ] [INFO ]  Estimator SVC already trained on abalone. Skipping it.
INFO:root:Estimator GaussianNaiveBayes already trained on abalone. Skipping it.
2018-09-19 21:50:25,722 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on abalone. Skipping it.
INFO:root:Estimato

2018-09-19 21:50:25,960 [MainThread  ] [INFO ]  Estimator SVC already trained on annealing. Skipping it.
INFO:root:Estimator GaussianNaiveBayes already trained on annealing. Skipping it.
2018-09-19 21:50:25,969 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on annealing. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already trained on annealing. Skipping it.
2018-09-19 21:50:25,976 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on annealing. Skipping it.
INFO:root:Estimator NeuralNetworkDeepClassifier already trained on annealing. Skipping it.
2018-09-19 21:50:25,980 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on annealing. Skipping it.
INFO:root:Estimator PassiveAggressiveClassifier already trained on annealing. Skipping it.
2018-09-19 21:50:25,984 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on annealing. Skipping it.
INFO:root:Estimator BaselineClassifier already tr

INFO:root:Estimator PassiveAggressiveClassifier already trained on balloons. Skipping it.
2018-09-19 21:50:26,296 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on balloons. Skipping it.
INFO:root:Estimator BaselineClassifier already trained on balloons. Skipping it.
2018-09-19 21:50:26,300 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on balloons. Skipping it.
INFO:root:Estimator RandomForestClassifier already trained on bank. Skipping it.
2018-09-19 21:50:26,343 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on bank. Skipping it.
INFO:root:Estimator BaggingClassifier already trained on bank. Skipping it.
2018-09-19 21:50:26,349 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on bank. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on bank. Skipping it.
2018-09-19 21:50:26,351 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on ban

2018-09-19 21:50:26,694 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on breast_cancer_wisc_diag. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on breast_cancer_wisc_diag. Skipping it.
2018-09-19 21:50:26,698 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on breast_cancer_wisc_diag. Skipping it.
INFO:root:Estimator SVC already trained on breast_cancer_wisc_diag. Skipping it.
2018-09-19 21:50:26,701 [MainThread  ] [INFO ]  Estimator SVC already trained on breast_cancer_wisc_diag. Skipping it.
INFO:root:Estimator GaussianNaiveBayes already trained on breast_cancer_wisc_diag. Skipping it.
2018-09-19 21:50:26,704 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on breast_cancer_wisc_diag. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already trained on breast_cancer_wisc_diag. Skipping it.
2018-09-19 21:50:26,707 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained 

INFO:root:Estimator SVC already trained on cardiotocography_10clases. Skipping it.
2018-09-19 21:50:26,962 [MainThread  ] [INFO ]  Estimator SVC already trained on cardiotocography_10clases. Skipping it.
INFO:root:Estimator GaussianNaiveBayes already trained on cardiotocography_10clases. Skipping it.
2018-09-19 21:50:26,965 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on cardiotocography_10clases. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already trained on cardiotocography_10clases. Skipping it.
2018-09-19 21:50:26,969 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on cardiotocography_10clases. Skipping it.
INFO:root:Estimator NeuralNetworkDeepClassifier already trained on cardiotocography_10clases. Skipping it.
2018-09-19 21:50:26,972 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on cardiotocography_10clases. Skipping it.
INFO:root:Estimator PassiveAggressiveClassifier already trained on cardiot

...

INFO:root:Estimator BaselineClassifier already trained on statlog_heart. Skipping it.
2018-09-19 21:50:33,132 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on statlog_heart. Skipping it.
INFO:root:Estimator RandomForestClassifier already trained on statlog_image. Skipping it.
2018-09-19 21:50:33,166 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on statlog_image. Skipping it.
INFO:root:Estimator BaggingClassifier already trained on statlog_image. Skipping it.
2018-09-19 21:50:33,169 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on statlog_image. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on statlog_image. Skipping it.
2018-09-19 21:50:33,177 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on statlog_image. Skipping it.
INFO:root:Estimator SVC already trained on statlog_image. Skipping it.
2018-09-19 21:50:33,180 [MainThread  ] [INFO ]  Estimator SVC alrea

INFO:root:Estimator GaussianNaiveBayes already trained on tic_tac_toe. Skipping it.
2018-09-19 21:50:33,790 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on tic_tac_toe. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already trained on tic_tac_toe. Skipping it.
2018-09-19 21:50:33,794 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on tic_tac_toe. Skipping it.
INFO:root:Estimator NeuralNetworkDeepClassifier already trained on tic_tac_toe. Skipping it.
2018-09-19 21:50:33,797 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on tic_tac_toe. Skipping it.
INFO:root:Estimator PassiveAggressiveClassifier already trained on tic_tac_toe. Skipping it.
2018-09-19 21:50:33,800 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on tic_tac_toe. Skipping it.
INFO:root:Estimator BaselineClassifier already trained on tic_tac_toe. Skipping it.
2018-09-19 21:50:33,804 [MainThread  ] [INFO ]  Estima

2018-09-19 21:50:34,122 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on vertebral_column_2clases. Skipping it.
INFO:root:Estimator BaselineClassifier already trained on vertebral_column_2clases. Skipping it.
2018-09-19 21:50:34,133 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on vertebral_column_2clases. Skipping it.
INFO:root:Estimator RandomForestClassifier already trained on vertebral_column_3clases. Skipping it.
2018-09-19 21:50:34,166 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on vertebral_column_3clases. Skipping it.
INFO:root:Estimator BaggingClassifier already trained on vertebral_column_3clases. Skipping it.
2018-09-19 21:50:34,169 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on vertebral_column_3clases. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on vertebral_column_3clases. Skipping it.
2018-09-19 21:50:34,182 [MainThread  ] [INFO ]  E

INFO:root:Estimator RandomForestClassifier already trained on wine. Skipping it.
2018-09-19 21:50:34,515 [MainThread  ] [INFO ]  Estimator RandomForestClassifier already trained on wine. Skipping it.
INFO:root:Estimator BaggingClassifier already trained on wine. Skipping it.
2018-09-19 21:50:34,519 [MainThread  ] [INFO ]  Estimator BaggingClassifier already trained on wine. Skipping it.
INFO:root:Estimator GradientBoostingClassifier already trained on wine. Skipping it.
2018-09-19 21:50:34,527 [MainThread  ] [INFO ]  Estimator GradientBoostingClassifier already trained on wine. Skipping it.
INFO:root:Estimator SVC already trained on wine. Skipping it.
2018-09-19 21:50:34,537 [MainThread  ] [INFO ]  Estimator SVC already trained on wine. Skipping it.
INFO:root:Estimator GaussianNaiveBayes already trained on wine. Skipping it.
2018-09-19 21:50:34,541 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on wine. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already 

INFO:root:Estimator GaussianNaiveBayes already trained on zoo. Skipping it.
2018-09-19 21:50:34,831 [MainThread  ] [INFO ]  Estimator GaussianNaiveBayes already trained on zoo. Skipping it.
INFO:root:Estimator BernoulliNaiveBayes already trained on zoo. Skipping it.
2018-09-19 21:50:34,834 [MainThread  ] [INFO ]  Estimator BernoulliNaiveBayes already trained on zoo. Skipping it.
INFO:root:Estimator NeuralNetworkDeepClassifier already trained on zoo. Skipping it.
2018-09-19 21:50:34,837 [MainThread  ] [INFO ]  Estimator NeuralNetworkDeepClassifier already trained on zoo. Skipping it.
INFO:root:Estimator PassiveAggressiveClassifier already trained on zoo. Skipping it.
2018-09-19 21:50:34,839 [MainThread  ] [INFO ]  Estimator PassiveAggressiveClassifier already trained on zoo. Skipping it.
INFO:root:Estimator BaselineClassifier already trained on zoo. Skipping it.
2018-09-19 21:50:34,842 [MainThread  ] [INFO ]  Estimator BaselineClassifier already trained on zoo. Skipping it.


### Step 5: Make predictions on the test sets

After the estimators are trained the user needs to use them in order to make predictions on the test sets which will be used subsequently for performing statistical tests.

In [8]:
orchest.predict_all(trained_models_dir='data/trained_models', estimators=estimators)

INFO:root:Predictions for NeuralNetworkDeepClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:34,890 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on abalone already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for RandomForestClassifier on abalone already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:34,898 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on abalone already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for SVC on abalone already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:34,902 [MainThread  ] [INFO ]  Predictions for SVC on abalone already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaggingClassifier on abalone already exist in the database. 

INFO:root:Predictions for BaselineClassifier on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,423 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on arrhythmia already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for GaussianNaiveBayes on arrhythmia already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,427 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on arrhythmia already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for NeuralNetworkDeepClassifier on audiology_std already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,458 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on audiology_std already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for RandomForestCl

INFO:root:Predictions for BernoulliNaiveBayes on balloons already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,679 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on balloons already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaselineClassifier on balloons already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,686 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on balloons already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for GaussianNaiveBayes on balloons already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,693 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on balloons already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for NeuralNetworkDeepClassifier on bank already exis

INFO:root:Predictions for BernoulliNaiveBayes on breast_cancer already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,991 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on breast_cancer already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaselineClassifier on breast_cancer already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:35,999 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on breast_cancer already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for GaussianNaiveBayes on breast_cancer already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:36,007 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on breast_cancer already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for NeuralNetworkDeepC

INFO:root:Predictions for GradientBoostingClassifier on breast_cancer_wisc_prog already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:36,311 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on breast_cancer_wisc_prog already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for PassiveAggressiveClassifier on breast_cancer_wisc_prog already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:36,315 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on breast_cancer_wisc_prog already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BernoulliNaiveBayes on breast_cancer_wisc_prog already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:36,320 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on breast_cancer_wisc_prog already exist in the databas

...

2018-09-19 21:50:46,234 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on titanic already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for NeuralNetworkDeepClassifier on trains already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,277 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on trains already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for RandomForestClassifier on trains already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,280 [MainThread  ] [INFO ]  Predictions for RandomForestClassifier on trains already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for SVC on trains already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,286 [MainThread  ] [INFO ]  Predictions for SVC on train

2018-09-19 21:50:46,480 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on vertebral_column_2clases already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for GaussianNaiveBayes on vertebral_column_2clases already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,485 [MainThread  ] [INFO ]  Predictions for GaussianNaiveBayes on vertebral_column_2clases already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for NeuralNetworkDeepClassifier on vertebral_column_3clases already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,515 [MainThread  ] [INFO ]  Predictions for NeuralNetworkDeepClassifier on vertebral_column_3clases already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for RandomForestClassifier on vertebral_column_3clases already exist in the database. Set o

2018-09-19 21:50:46,714 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on waveform already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BernoulliNaiveBayes on waveform already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,717 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on waveform already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaselineClassifier on waveform already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,725 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on waveform already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for GaussianNaiveBayes on waveform already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:46,730 [MainThread  ] [INFO ]  Predictions for G

INFO:root:Predictions for PassiveAggressiveClassifier on wine_quality_red already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,020 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on wine_quality_red already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BernoulliNaiveBayes on wine_quality_red already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,043 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on wine_quality_red already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaselineClassifier on wine_quality_red already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,058 [MainThread  ] [INFO ]  Predictions for BaselineClassifier on wine_quality_red already exist in the database. Set override to True if you wish replace them.
INFO:roo

INFO:root:Predictions for GradientBoostingClassifier on zoo already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,295 [MainThread  ] [INFO ]  Predictions for GradientBoostingClassifier on zoo already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for PassiveAggressiveClassifier on zoo already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,299 [MainThread  ] [INFO ]  Predictions for PassiveAggressiveClassifier on zoo already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BernoulliNaiveBayes on zoo already exist in the database. Set override to True if you wish replace them.
2018-09-19 21:50:47,310 [MainThread  ] [INFO ]  Predictions for BernoulliNaiveBayes on zoo already exist in the database. Set override to True if you wish replace them.
INFO:root:Predictions for BaselineClassifier on zoo already exist in t

### Step 6: Analyze the results

The last step in the pipeline is to analyze the results of the experiments.

The `AnalyseResults` class takes as inputs the two database files and the loss metric that will be used to compute the prediction errors.

The `prediction_errors()` method retuns two sets of results: `errors_per_estimator` dictionary which is used subsequently in further statistical tests and `errors_per_dataset_per_estimator_df` which is a dataframe with the loss of each estimator on each dataset which can be examined directly by the user. 

In [9]:
analyze = AnalyseResults(hdf5_output_io=out_io, 
                         hdf5_input_io=input_io,
                         input_h5_original_datasets_group='delgado_datasets/', 
                         output_h5_predictions_group='experiments/predictions/')
score_accuracy = ScoreAccuracy()


(errors_per_estimator, 
 errors_per_dataset_per_estimator, 
 errors_per_dataset_per_estimator_df) = analyze.prediction_errors(score_accuracy, estimators)

Below we show the results of the various statistical tests that are supported by MLAUT

#### t-test

In [10]:
t_test, t_test_df = analyze.t_test(errors_per_estimator)
t_test_df

BaggingClassifier        BaselineClassifier        \
                                       t_stat  p_val             t_stat p_val   
BaggingClassifier                       0.000  1.000             17.370   0.0   
BaselineClassifier                    -17.370  0.000              0.000   1.0   
BernoulliNaiveBayes                    -5.613  0.000             12.267   0.0   
GaussianNaiveBayes                     -6.226  0.000              9.978   0.0   
GradientBoostingClassifier             -1.248  0.213             15.541   0.0   
NeuralNetworkDeepClassifier            -6.241  0.000              8.572   0.0   
PassiveAggressiveClassifier            -2.836  0.005             14.077   0.0   
RandomForestClassifier                  0.839  0.402             18.455   0.0   
SVC                                    -0.088  0.930             17.435   0.0   

                            BernoulliNaiveBayes        GaussianNaiveBayes  \
                                         t_stat  p_val             t_stat   
BaggingClassifier                         5.613  0.000              6.226   
BaselineClassifier                      -12.267  0.000             -9.978   
BernoulliNaiveBayes                       0.000  1.000              1.274   
GaussianNaiveBayes                       -1.274  0.204              0.000   
GradientBoostingClassifier                4.077  0.000              4.865   
NeuralNetworkDeepClassifier              -1.789  0.075             -0.591   
PassiveAggressiveClassifier               2.485  0.014              3.447   
RandomForestClassifier                    6.594  0.000              7.078   
SVC                                       5.587  0.000              6.202   

                                   GradientBoostingClassifier         \
                             p_val                     t_stat  p_val   
BaggingClassifier            0.000                      1.248  0.213   
BaselineClassifier           0.000                    -15.541  0.000   
BernoulliNaiveBayes          0.204                     -4.077  0.000   
GaussianNaiveBayes           1.000                     -4.865  0.000   
GradientBoostingClassifier   0.000                      0.000  1.000   
NeuralNetworkDeepClassifier  0.555                     -5.033  0.000   
PassiveAggressiveClassifier  0.001                     -1.514  0.131   
RandomForestClassifier       0.000                      2.069  0.040   
SVC                          0.000                      1.177  0.240   

                            NeuralNetworkDeepClassifier         \
                                                 t_stat  p_val   
BaggingClassifier                                 6.241  0.000   
BaselineClassifier                               -8.572  0.000   
BernoulliNaiveBayes                               1.789  0.075   
GaussianNaiveBayes                                0.591  0.555   
GradientBoostingClassifier                        5.033  0.000   
NeuralNetworkDeepClassifier                       0.000  1.000   
PassiveAggressiveClassifier                       3.749  0.000   
RandomForestClassifier                            6.990  0.000   
SVC                                               6.215  0.000   

                            PassiveAggressiveClassifier         \
                                                 t_stat  p_val   
BaggingClassifier                                 2.836  0.005   
BaselineClassifier                              -14.077  0.000   
BernoulliNaiveBayes                              -2.485  0.014   
GaussianNaiveBayes                               -3.447  0.001   
GradientBoostingClassifier                        1.514  0.131   
NeuralNetworkDeepClassifier                      -3.749  0.000   
PassiveAggressiveClassifier                       0.000  1.000   
RandomForestClassifier                            3.693  0.000   
SVC                                               2.782  0.006   

                            RandomForestC

#### sign test

In [11]:
sign_test, sign_test_df = analyze.sign_test(errors_per_estimator)
sign_test_df

BaggingClassifier        BaselineClassifier        \
                                       t_stat  p_val             t_stat p_val   
BaggingClassifier                       0.000  1.000             11.913   0.0   
BaselineClassifier                    -11.913  0.000              0.000   1.0   
BernoulliNaiveBayes                    -5.687  0.000              9.995   0.0   
GaussianNaiveBayes                     -6.021  0.000              8.812   0.0   
GradientBoostingClassifier             -1.120  0.263             11.253   0.0   
NeuralNetworkDeepClassifier            -5.698  0.000              7.778   0.0   
PassiveAggressiveClassifier            -2.753  0.006             10.776   0.0   
RandomForestClassifier                  0.879  0.380             12.224   0.0   
SVC                                    -0.152  0.880             11.940   0.0   

                            BernoulliNaiveBayes        GaussianNaiveBayes  \
                                         t_stat  p_val             t_stat   
BaggingClassifier                         5.687  0.000              6.021   
BaselineClassifier                       -9.995  0.000             -8.812   
BernoulliNaiveBayes                       0.000  1.000              0.788   
GaussianNaiveBayes                       -0.788  0.431              0.000   
GradientBoostingClassifier                4.260  0.000              4.742   
NeuralNetworkDeepClassifier              -0.856  0.392             -0.151   
PassiveAggressiveClassifier               2.865  0.004              3.351   
RandomForestClassifier                    6.513  0.000              6.788   
SVC                                       5.671  0.000              5.994   

                                   GradientBoostingClassifier         \
                             p_val                     t_stat  p_val   
BaggingClassifier            0.000                      1.120  0.263   
BaselineClassifier           0.000                    -11.253  0.000   
BernoulliNaiveBayes          0.431                     -4.260  0.000   
GaussianNaiveBayes           1.000                     -4.742  0.000   
GradientBoostingClassifier   0.000                      0.000  1.000   
NeuralNetworkDeepClassifier  0.880                     -4.597  0.000   
PassiveAggressiveClassifier  0.001                     -1.608  0.108   
RandomForestClassifier       0.000                      1.845  0.065   
SVC                          0.000                      0.912  0.362   

                            NeuralNetworkDeepClassifier         \
                                                 t_stat  p_val   
BaggingClassifier                                 5.698  0.000   
BaselineClassifier                               -7.778  0.000   
BernoulliNaiveBayes                               0.856  0.392   
GaussianNaiveBayes                                0.151  0.880   
GradientBoostingClassifier                        4.597  0.000   
NeuralNetworkDeepClassifier                       0.000  1.000   
PassiveAggressiveClassifier                       3.291  0.001   
RandomForestClassifier                            6.444  0.000   
SVC                                               5.684  0.000   

                            PassiveAggressiveClassifier         \
                                                 t_stat  p_val   
BaggingClassifier                                 2.753  0.006   
BaselineClassifier                              -10.776  0.000   
BernoulliNaiveBayes                              -2.865  0.004   
GaussianNaiveBayes                               -3.351  0.001   
GradientBoostingClassifier                        1.608  0.108   
NeuralNetworkDeepClassifier                      -3.291  0.001   
PassiveAggressiveClassifier                       0.000  1.000   
RandomForestClassifier                            3.610  0.000   
SVC                                               2.732  0.006   

                            RandomForestC

#### t-test with bonferroni correction

In [12]:
t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(errors_per_estimator)
t_test_bonferroni_df

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,False,True,True,True,False,True,False,False,False
BaselineClassifier,True,False,True,True,True,True,True,True,True
BernoulliNaiveBayes,True,True,False,False,True,False,False,True,True
GaussianNaiveBayes,True,True,False,False,True,False,False,True,True
GradientBoostingClassifier,False,True,True,True,False,True,False,False,False
NeuralNetworkDeepClassifier,True,True,False,False,True,False,True,True,True
PassiveAggressiveClassifier,False,True,False,False,False,True,False,True,False
RandomForestClassifier,False,True,True,True,False,True,True,False,False
SVC,False,True,True,True,False,True,False,False,False


#### Wilcoxon test

In [13]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(errors_per_estimator)
wilcoxon_test_df

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/scipy/stats/morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/scipy/stats/morestats.py:2410: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se


BaggingClassifier        BaselineClassifier        \
                                    statistic  p_val          statistic p_val   
BaggingClassifier                         0.0    NaN               10.0   0.0   
BaselineClassifier                       10.0  0.000                0.0   NaN   
BernoulliNaiveBayes                     621.0  0.000               50.0   0.0   
GaussianNaiveBayes                      512.5  0.000              458.0   0.0   
GradientBoostingClassifier             1289.0  0.000               26.5   0.0   
NeuralNetworkDeepClassifier             619.0  0.000              231.0   0.0   
PassiveAggressiveClassifier            1173.0  0.000                6.5   0.0   
RandomForestClassifier                 1167.0  0.000                0.0   0.0   
SVC                                    2762.5  0.949                0.0   0.0   

                            BernoulliNaiveBayes        GaussianNaiveBayes  \
                                      statistic  p_val          statistic   
BaggingClassifier                         621.0  0.000              512.5   
BaselineClassifier                         50.0  0.000              458.0   
BernoulliNaiveBayes                         0.0    NaN             2625.5   
GaussianNaiveBayes                       2625.5  0.088                0.0   
GradientBoostingClassifier               1204.5  0.000             1151.5   
NeuralNetworkDeepClassifier              2847.0  0.133             3085.0   
PassiveAggressiveClassifier              1454.0  0.000             1094.5   
RandomForestClassifier                    165.5  0.000              191.5   
SVC                                       392.5  0.000              224.0   

                                   GradientBoostingClassifier         \
                             p_val                  statistic  p_val   
BaggingClassifier            0.000                     1289.0  0.000   
BaselineClassifier           0.000                       26.5  0.000   
BernoulliNaiveBayes          0.088                     1204.5  0.000   
GaussianNaiveBayes             NaN                     1151.5  0.000   
GradientBoostingClassifier   0.000                        0.0    NaN   
NeuralNetworkDeepClassifier  0.586                      861.0  0.000   
PassiveAggressiveClassifier  0.000                     2572.5  0.033   
RandomForestClassifier       0.000                      645.5  0.000   
SVC                          0.000                     1965.0  0.001   

                            NeuralNetworkDeepClassifier         \
                                              statistic  p_val   
BaggingClassifier                                 619.0  0.000   
BaselineClassifier                                231.0  0.000   
BernoulliNaiveBayes                              2847.0  0.133   
GaussianNaiveBayes                               3085.0  0.586   
GradientBoostingClassifier                        861.0  0.000   
NeuralNetworkDeepClassifier                         0.0    NaN   
PassiveAggressiveClassifier                      1471.0  0.000   
RandomForestClassifier                            269.0  0.000   
SVC                                               263.0  0.000   

                            PassiveAggressiveClassifier         \
                                              statistic  p_val   
BaggingClassifier                                1173.0  0.000   
BaselineClassifier                                  6.5  0.000   
BernoulliNaiveBayes                              1454.0  0.000   
GaussianNaiveBayes                               1094.5  0.000   
GradientBoostingClassifier                       2572.5  0.033   
NeuralNetworkDeepClassifier                      1471.0  0.000   
PassiveAggressiveClassifier                         0.0    NaN   
RandomForestClassifier                            527.0  0.000   
SVC                                               726.5  0.000   

                            RandomForestC

#### Friedman test

In [14]:
friedman_test, friedman_test_df = analyze.friedman_test(errors_per_estimator)
friedman_test_df

,statistic,p_value
0,506.211,0.0


#### Nemenyi test

In [15]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,-1.000,0.0,0.001,0.000,0.997,0.000,0.593,1.000,1.000
BaselineClassifier,0.000,-1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000
BernoulliNaiveBayes,0.001,0.0,-1.000,1.000,0.042,1.000,0.590,0.000,0.001
GaussianNaiveBayes,0.000,0.0,1.000,-1.000,0.007,1.000,0.271,0.000,0.000
GradientBoostingClassifier,0.997,0.0,0.042,0.007,-1.000,0.006,0.978,0.902,0.998
NeuralNetworkDeepClassifier,0.000,0.0,1.000,1.000,0.006,-1.000,0.260,0.000,0.000
PassiveAggressiveClassifier,0.593,0.0,0.590,0.271,0.978,0.260,-1.000,0.204,0.636
RandomForestClassifier,1.000,0.0,0.000,0.000,0.902,0.000,0.204,-1.000,1.000
SVC,1.000,0.0,0.001,0.000,0.998,0.000,0.636,1.000,-1.000


In [16]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,-1.000,0.0,0.001,0.000,0.997,0.000,0.593,1.000,1.000
BaselineClassifier,0.000,-1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000
BernoulliNaiveBayes,0.001,0.0,-1.000,1.000,0.042,1.000,0.590,0.000,0.001
GaussianNaiveBayes,0.000,0.0,1.000,-1.000,0.007,1.000,0.271,0.000,0.000
GradientBoostingClassifier,0.997,0.0,0.042,0.007,-1.000,0.006,0.978,0.902,0.998
NeuralNetworkDeepClassifier,0.000,0.0,1.000,1.000,0.006,-1.000,0.260,0.000,0.000
PassiveAggressiveClassifier,0.593,0.0,0.590,0.271,0.978,0.260,-1.000,0.204,0.636
RandomForestClassifier,1.000,0.0,0.000,0.000,0.902,0.000,0.204,-1.000,1.000
SVC,1.000,0.0,0.001,0.000,0.998,0.000,0.636,1.000,-1.000


In [17]:
pd.set_option('display.max_rows', 5000)
errors_per_dataset_per_estimator_df

loss  std_error
abalone                        BaggingClassifier            0.37708    0.01305
                               BaselineClassifier           0.67440    0.01262
                               BernoulliNaiveBayes          0.44888    0.01339
                               GaussianNaiveBayes           0.44017    0.01337
                               GradientBoostingClassifier   0.38869    0.01313
                               NeuralNetworkDeepClassifier  0.37273    0.01302
                               PassiveAggressiveClassifier  0.37346    0.01303
                               RandomForestClassifier       0.36476    0.01296
                               SVC                          0.36186    0.01294
acute_inflammation             BaggingClassifier            0.00000    0.00000
                               BaselineClassifier           0.52500    0.07896
                               BernoulliNaiveBayes          0.12500    0.05229
                               GaussianNaiveBayes           0.17500    0.06008
                               GradientBoostingClassifier   0.00000    0.00000
                               NeuralNetworkDeepClassifier  0.17500    0.06008
                               PassiveAggressiveClassifier  0.00000    0.00000
                               RandomForestClassifier       0.00000    0.00000
                               SVC                          0.00000    0.00000
acute_nephritis                BaggingClassifier            0.00000    0.00000
                               BaselineClassifier           0.55000    0.07866
                               BernoulliNaiveBayes          0.02500    0.02469
                               GaussianNaiveBayes           0.02500    0.02469
                               GradientBoostingClassifier   0.00000    0.00000
                               NeuralNetworkDeepClassifier  0.00000    0.00000
                               PassiveAggressiveClassifier  0.00000    0.00000
                               RandomForestClassifier       0.00000    0.00000
                               SVC                          0.00000    0.00000
adult                          BaggingClassifier            0.14772    0.00279
                               BaselineClassifier           0.36053    0.00378
                               BernoulliNaiveBayes          0.19612    0.00313
                               GaussianNaiveBayes           0.19121    0.00310
                               GradientBoostingClassifier   0.13314    0.00268
                               NeuralNetworkDeepClassifier  0.15380    0.00284
                               PassiveAggressiveClassifier  0.17856    0.00302
                               RandomForestClassifier       0.13780    0.00271
                               SVC                          0.15275    0.00283
annealing                      BaggingClassifier            0.04714    0.01230
                               BaselineClassifier           0.39394    0.02835
                               BernoulliNaiveBayes          0.15825    0.02118
                               GaussianNaiveBayes           0.47138    0.02897
                               GradientBoostingClassifier   0.05051    0.01271
                               NeuralNetworkDeepClassifier  0.24579    0.02498
                               PassiveAggressiveClassifier  0.19192    0.02285
                               RandomForestClassifier       0.04377    0.01187
                               SVC                          0.11111    0.01824
arrhythmia                     BaggingClassifier            0.25333    0.03551
                               BaselineClassifier           0.70667    0.03717
                               BernoulliNaiveBayes          0.42000    0.04030
                               GaussianNaiveBayes           0.84667    0.02942
                               GradientBoostingClassifier   0.29333    0.03717
                               Neu